Notes for me:

other options aside from llama 3.1:

https://medium.com/geekculture/how-to-label-text-data-using-llms-f0ffc3fcd168
https://labelyourdata.com/articles/llm-data-labeling

Cleaning process:

- action as a list

- everything included if not there add empty string

- If any field that is not supposed to be there has been added, remove it.

Check ratings and radius from hotels

## GPT

In [1]:
import openai

In [ ]:
# Make sure to set your API key
key = "MY_KEY"

In [3]:
models = openai.Model.list()
print(models)


{
  "object": "list",
  "data": [
    {
      "id": "gpt-4o-mini",
      "object": "model",
      "created": 1721172741,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-mini-2024-07-18",
      "object": "model",
      "created": 1721172717,
      "owned_by": "system"
    },
    {
      "id": "dall-e-2",
      "object": "model",
      "created": 1698798177,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-ada-002",
      "object": "model",
      "created": 1671217299,
      "owned_by": "openai-internal"
    },
    {
      "id": "gpt-4-1106-preview",
      "object": "model",
      "created": 1698957206,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o",
      "object": "model",
      "created": 1715367049,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-2024-08-06",
      "object": "model",
      "created": 1722814719,
      "owned_by": "system"
    },
    {
      "id": "text-embedding-3-large",
      "object": "model",
      "created

In [14]:
import openai
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()


In [96]:
prompt = """
Generate a JSON object where:
1. "input" is a single string where a user requests information **on different locations** for a **flight, hotel, and activities**. The request involves travel from one location to another, specifying a departure date, and optionally includes details such as the number of passengers, flight class, non-stop preference, hotel ratings, amenities, or other preferences. The input is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights", "search_hotels", "activities"].
   - The "action_input" field includes:
     - "origin": A 3-letter city code for the departure location or airport 3-letter code if specified.
     - "destination": A 3-letter city code for the arrival location or airport 3-letter code if specified.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields are included only if explicitly mentioned in the input and should be excluded otherwise:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer representing the number of children.
       - "infants": An integer representing the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
       - "ratings": An integer for hotel ratings (out of 5).
       - "amenities": A list of amenities required by the user. Supported values: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
     - "max": Always set to 5.

3. Ensure the input always requests all three elements (flights, hotels, and activities). Avoid generating inputs that omit any of these.
Generate ONE EXAMPLE PER RESPONSE object. Use varied phrasing and combinations of optional fields to add diversity to the input while adhering to the guidelines.
"""


In [98]:
import asyncio
import aiohttp
import json

# async def generate_example(session, prompt, temperature=1):
#     retries = 3
#     backoff_factor = 0.5
#     for attempt in range(retries):
#         try:
#             response = await session.post(
#                 'https://api.openai.com/v1/chat/completions',
#                 headers={'Authorization': f'Bearer {key}'},  # Replace YOUR_API_KEY with your actual OpenAI API key
#                 json={
#                     'model': "chatgpt-4o-latest",
#                     'messages': [
#                         {"role": "system", "content": "You are a data generator."},
#                         {"role": "user", "content": prompt}
#                     ],
#                     'temperature': temperature,
#                     'max_tokens': 350
#                 }
#             )
#             data = await response.json()
#             content = data['choices'][0]['message']['content'].strip()
#             if content.startswith("{") and content.endswith("}"):
#                 return json.loads(content)
#         except Exception as e:
#             if attempt < retries - 1:
#                 await asyncio.sleep(backoff_factor * (2 ** attempt))
#             else:
#                 print(f"Final retry failed: {e}")
#     return None

# async def generate_example(session, prompt, temperature=1):
#     retries = 3
#     backoff_factor = 0.5
#     for attempt in range(retries):
#         try:
#             response = await session.post(
#                 'https://api.openai.com/v1/chat/completions',
#                 headers={'Authorization': f'Bearer {key}'},  # Ensure key is correctly set here
#                 json={
#                     'model': "chatgpt-4o-latest",
#                     'messages': [
#                         {"role": "system", "content": "You are a data generator."},
#                         {"role": "user", "content": prompt}
#                     ],
#                     'temperature': temperature,
#                     'max_tokens': 500
#                 }
#             )
#             if response.status == 200:
#                 data = await response.json()
#                 content = data['choices'][0]['message']['content'].strip()
#                 print(f"API Content: {content}")  # Debug: Check what content is returned
#                 if content.startswith("{") and content.endswith("}"):
#                     return json.loads(content)
#                 else:
#                     print("Content not in JSON format")
#             else:
#                 print(f"HTTP Error {response.status}: {await response.text()}")
#         except Exception as e:
#             print(f"Attempt {attempt + 1} failed: {e}")
#             if attempt < retries - 1:
#                 await asyncio.sleep(backoff_factor * (2 ** attempt))
#     return None

import asyncio
import aiohttp
import json

async def generate_example(session, prompt, temperature=1):
    retries = 3
    backoff_factor = 0.5
    for attempt in range(retries):
        try:
            response = await session.post(
                'https://api.openai.com/v1/chat/completions',
                headers={'Authorization': f'Bearer {key}'},  # Replace YOUR_API_KEY with your actual OpenAI API key
                json={
                    'model': "chatgpt-4o-latest", 
                    'messages': [
                        {"role": "system", "content": "You are a data generator."},
                        {"role": "user", "content": prompt}
                    ],
                    'temperature': temperature,
                    'max_tokens': 750
                }
            )
            if response.status == 200:
                data = await response.json()
                content = data['choices'][0]['message']['content'].strip()
                # Remove any non-JSON valid characters and the word "json"
                content = content.replace('`', '').replace('json', '').replace('\n', '').strip()
                print(f"Cleaned Content: {content}")  # Debug: Show cleaned content
                if content.startswith("{") and content.endswith("}"):
                    return json.loads(content)
                else:
                    print("Content not in JSON format")
            else:
                print(f"HTTP Error {response.status}: {await response.text()}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                await asyncio.sleep(backoff_factor * (2 ** attempt))
    return None

# Main and other supporting functions remain the same.



async def generate_multiple_examples(prompt, num_examples=90, temperature=1):
    examples = []
    async with aiohttp.ClientSession() as session:
        tasks = [asyncio.create_task(generate_example(session, prompt, temperature)) for _ in range(num_examples)]
        for i, task in enumerate(asyncio.as_completed(tasks), 1):
            result = await task
            if result:
                examples.append(result)
            else:
                print(f"Example {i}: Received None")
    return examples

async def main():
    examples = await generate_multiple_examples(prompt, num_examples=90)
    save_to_json(examples, 'Flight_Hotel_Activ1.json')  # Save examples to a JSON file

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4) 

# Start the async process
if __name__ == '__main__':
    asyncio.run(main())

Cleaned Content: {  "input": "I’d like to travel from New York (JFK) to Paris (CDG) on March 15, 2024. It’s for two adults and one child. I need business class tickets and prefer non-stop flights. For hotels, I’m looking for something luxurious with at least 5 stars, a swimming pool, and a spa. Can you also suggest some activities there?",  "output": {    "action": ["search_flights", "search_hotels", "activities"],    "action_input": {      "origin": "JFK",      "destination": "CDG",      "departure_date": "2024-03-15",      "adults": 2,      "children": 1,      "class": "BUSINESS",      "nonStop": true,      "ratings": 5,      "amenities": ["SWIMMING_POOL", "SPA"],      "max": 5    }  }}
Cleaned Content: {  "input": "Can you help me plan a trip from Los Angeles (LAX) to New York City (JFK) on April 10, 2024? I need a non-stop flight for 2 adults and a child in business class. I’ll also need a 4-star hotel with a swimming pool and wifi. Lastly, suggest some activities in the area for s

In [100]:
# read json file and check length
import json

# read json file and check length
with open('Flight_Hotel_Activ.json') as f:
    data = json.load(f)
    print(len(data))

100


In [99]:
# join Flight_hotel_2.json and Flight_hotel.json
import json

with open('Flight_Hotel_Activ.json') as f:
    data1 = json.load(f)

with open('Flight_Hotel_Activ1.json') as f:
    data2 = json.load(f)

data1.extend(data2)

with open('Flight_Hotel_Activ.json', 'w') as f:
    json.dump(data1, f, indent=4)

# Flights: ready

In [ ]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a flight from one location to another and asks about a specific day, with additional information sometimes added like number of passengers, class, if only non-stop, etc. This is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights"].
   - The "action_input" field contains:
     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.
     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields should be included if mentioned and returned as an empty string if not mentioned:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer for the number of children.
       - "infants": An integer for the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
     - "max": Always set to 5.

3. Generate one example per response object. Be variable in the input format and the output fields following the guidelines above.
"""

# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.1", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Example 1:
{
    "input": "flight from JFK to LAX on 2022-07-25",
    "output": {
        "action": [
            "search_flights"
        ],
        "action_input": {
            "origin": "JFK",
            "destination": "LAX",
            "departure_date": "2022-07-25",
            "adults": "1",
            "max": "5"
        }
    }
}
Example 2:
{
    "input": "What is the cheapest flight from London to Paris on May 15th for a family of three, with two adults and one child?",
    "output": {
        "action": [
            "search_flights"
        ],
        "action_input": {
            "origin": "LON",
            "destination": "PAR",
            "departure_date": "2023-05-15",
            "adults": 2,
            "children": 1,
            "max": 5
        }
    }
}
Example 3:
{
    "input": "flight from New York to London on May 15th with a child and only non-stop",
    "output": {
        "action": [
            "search_flights"
        ],
        "action_input": {
        

# Hotels:

In [ ]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information about hotels in a specific location. Additional preferences or requirements (such as star ratings or amenities) may or may not be mentioned. The input is provided as plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_hotels"].
   - The "action_input" field includes:
     - "cityCode": A 3-letter city code for the requested location.
     - Optional fields to include only if explicitly mentioned in the user request:
       - "ratings": An integer representing the hotel ratings (out of 5), included only if ratings are mentioned.
       - "amenities": A list of amenities required by the user, included only if amenities are mentioned. Supported values: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
   - If ratings or amenities are not mentioned, these fields should be excluded from the output entirely (not set to empty or null).

3. Generate one example per response object, with variations in the input phrasing and combinations of optional fields. Ensure no hallucination of fields or values that are not explicitly requested in the "input."
"""


# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.1", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Example 1:
{
    "input": "I'm looking for a hotel with four stars in Madrid.",
    "output": {
        "action": [
            "search_hotels"
        ],
        "action_input": {
            "cityCode": "MAD",
            "ratings": 4
        }
    }
}
Example 2:
{
    "input": "I'm looking for hotels in Paris",
    "output": {
        "action": [
            "search_hotels"
        ],
        "action_input": {
            "cityCode": "PAR"
        }
    }
}
Example 3:
{
    "input": "I'm looking for a hotel in New York City.",
    "output": {
        "action": [
            "search_hotels"
        ],
        "action_input": {
            "cityCode": "NYC"
        }
    }
}
Example 4:
{
    "input": "Show me hotels in Paris",
    "output": {
        "action": [
            "search_hotels"
        ],
        "action_input": {
            "cityCode": "PAR"
        }
    }
}
Example 5:
{
    "input": "Hotels near Eiffel Tower",
    "output": {
        "action": [
            "search_hot

# Activities:

In [4]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a activities or things to do in one location (change it everytime).
2. "output" contains the corresponding Amadeus API call in JSON format, do not change the field names:
   - "action": it is always ["activities"].
   - "destination": A 3-letter city code for the location requested.

3. Generate one example per response object. Be variable in the location/cityCode.
"""

# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.1", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Example 1:
{
    "input": "Recommend things to do in Paris",
    "output": {
        "action": [
            "activities"
        ],
        "destination": "PAR"
    }
}
Example 2:
{
    "input": "I want to know what I can do in Paris",
    "output": {
        "action": [
            "activities"
        ],
        "destination": "PAR"
    }
}
Example 3:
{
    "input": "Recommend things to do in New York",
    "output": {
        "action": [
            "activities"
        ],
        "destination": "NYC"
    }
}
Example 4:
{
    "input": "What are some things to do in Paris?",
    "output": {
        "action": [
            "activities"
        ],
        "destination": "PAR"
    }
}
Example 5:
{
    "input": "Show me activities and things to do in Paris",
    "output": {
        "action": [
            "activities"
        ],
        "destination": "PAR"
    }
}
Example 6:
{
    "input": "I'm looking for fun activities in Tokyo",
    "output": {
        "action": [
            "activ

# Flight + Hotel:

In [ ]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a hotel and a flight from one location to another and asks about a specific dayAdditional preferences or requirements (such as number of people, non-stop flights only, max prices, return dates, star ratings or amenities) may or may not be mentioned. The input is provided as plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights", "search_hotels"].
   - The "action_input" field contains:
     - "origin": A 3-letter city code for the departure location or airport 3 letter code if mentioned.
     - "destination": A 3-letter city code for the arrival location or airport 3 letter code if mentioned.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields should be included if mentioned and returned as an empty string if not mentioned:
       - "return_date": Only if mentioned in input it is a date in "YYYY-MM-DD" format for the return flight.
       - "children": Only if mentioned in input it is  an integer for the number of children.
       - "infants": Only if mentioned in input it is an integer for the number of infants.
       - "class": Only if mentioned in input it is one of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": Only if mentioned in input it is a boolean indicating whether only non-stop flights are desired.
       - "maxPrice": Only if mentioned in input it is an integer specifying the maximum price.
       - "ratings": Only if mentioned in input it is an int with the ratings of the hotel (out of 5).
       - "amenities" : Only if mentioned in input it is a list of amenities required by user, change between: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
     - "max": Always set to 5.

3. Generate one example per response object. Be variable in the input format and the output fields following the guidelines above.
"""

# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.1", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Example 1:
{
    "input": "Book me a hotel and flight from New York JFK on December 15th, 2022, for two adults with non-stop flights only, maximum price of $1,000 and return flight on January 5th, 2023. The hotel should have at least three stars.",
    "output": {
        "action": [
            "search_flights",
            "search_hotels"
        ],
        "action_input": {
            "origin": "JFK",
            "destination": "LAX",
            "departure_date": "2022-12-15",
            "adults": 2,
            "nonStop": true,
            "maxPrice": 1000,
            "return_date": "2023-01-05"
        }
    }
}
Example 2:
{
    "input": "Book a flight from NYC to LAX on August 15th for two people with one infant, non-stop only. And I need 4-star hotel at LAX.",
    "output": {
        "action": [
            "search_flights",
            "search_hotels"
        ],
        "action_input": {
            "origin": "NYC",
            "destination": "LAX",
            "departure_d

# Hotel + Activities:

In [13]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information about requests information for activities and hotels in a specific location. Additional preferences or requirements (such as star ratings or amenities) may or may not be mentioned. The input is provided as plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_hotels", "activities"].
   - The "action_input" field includes:
     - "destination": A 3-letter city code for the requested location.
     - Optional fields to include only if explicitly mentioned in the user request:
       - "ratings": Only if mentioned in input it is an integer with the ratings of the hotel (out of 5).
       - "amenities": Only if mentioned in input it is a list of amenities required by the user, included only if amenities are mentioned. Supported values: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
   - If ratings or amenities are not mentioned, these fields should be excluded from the output entirely (not set to empty or null).

3. Generate one example per response object, with variations in the input phrasing and combinations of optional fields. Ensure no hallucination of fields or values that are not explicitly requested in the "input."
"""

# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.1", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Example 1:
{
    "input": "What activities and hotels do you have available for New York?",
    "output": {
        "action": [
            "search_hotels",
            "activities"
        ],
        "action_input": {
            "destination": "NYC"
        }
    }
}
Example 2:
{
    "input": "I'd like to know more about activities and hotels in Paris.",
    "output": {
        "action": [
            "search_hotels",
            "activities"
        ],
        "action_input": {
            "destination": "PAR"
        }
    }
}
Example 3:
{
    "input": "I'm looking for activities and hotels in Paris.",
    "output": {
        "action": [
            "search_hotels",
            "activities"
        ],
        "action_input": {
            "destination": "PAR"
        }
    }
}
Example 4:
{
    "input": "What activities and hotels are available in New York City? I prefer hotels with a fitness center.",
    "output": {
        "action": [
            "search_hotels",
            "act

# Flight + Hotel + Activities:

In [ ]:
from langchain_community.llms import ollama
import json

# Define the system prompt
SYSTEM_PROMPT = """
Generate a JSON object where:
1. "input" is a single string where a user requests information for a **flight, hotel, and activities**. The request involves travel from one location to another, specifying a departure date, and optionally includes details such as the number of passengers, flight class, non-stop preference, hotel ratings, amenities, or other preferences. The input is provided in plain text, not JSON.
2. "output" contains the corresponding Amadeus API call in JSON format:
   - The "action" field is always ["search_flights", "search_hotels", "activities"].
   - The "action_input" field includes:
     - "origin": A 3-letter city code for the departure location or airport 3-letter code if specified.
     - "destination": A 3-letter city code for the arrival location or airport 3-letter code if specified.
     - "departure_date": A date in "YYYY-MM-DD" format.
     - "adults": An integer representing the number of adults (default is 1).
     - Optional fields are included only if explicitly mentioned in the input and should be excluded otherwise:
       - "return_date": A date in "YYYY-MM-DD" format for the return flight.
       - "children": An integer representing the number of children.
       - "infants": An integer representing the number of infants.
       - "class": One of "ECONOMY", "PREMIUM_ECONOMY", "BUSINESS", or "FIRST".
       - "nonStop": A boolean indicating whether only non-stop flights are desired.
       - "maxPrice": An integer specifying the maximum price.
       - "ratings": An integer for hotel ratings (out of 5).
       - "amenities": A list of amenities required by the user. Supported values: SWIMMING_POOL, SPA, FITNESS_CENTER, AIR_CONDITIONING, PETS_ALLOWED, WIFI.
     - "max": Always set to 5.

3. Ensure the input always requests all three elements (flights, hotels, and activities). Avoid generating inputs that omit any of these.
4. Generate one example per response object. Use varied phrasing and combinations of optional fields to add diversity to the input while adhering to the guidelines.
"""


# Initialize the Ollama LLM
llm = ollama.Ollama(model="llama3.1", format="json", verbose=False)

# Generate 10 examples with varying temperatures
examples = []
for i in range(10):
    # Update temperature for diversity
    temperature = 1

    # Generate a response using the fixed system prompt
    response = llm.invoke(
        SYSTEM_PROMPT,
        config={"temperature": temperature}
    )
    
    try:
        # Parse the response and append it to the examples list
        example = json.loads(response)
        examples.append(example)
    except json.JSONDecodeError:
        print(f"Failed to decode response at iteration {i + 1}: {response}")

# Print all generated examples
for idx, example in enumerate(examples):
    print(f"Example {idx + 1}:")
    print(json.dumps(example, indent=4))

Example 1:
{
    "input": "What are the prices for a flight from LAX to JFK on 2023-03-15, with return on 2023-03-20? I'd also like to book a hotel in New York City with at least a 4-star rating and ensure it has air conditioning. What activities can be found around that area?",
    "output": {
        "action": [
            "search_flights",
            "search_hotels",
            "activities"
        ],
        "action_input": {
            "origin": "LAX",
            "destination": "JFK",
            "departure_date": "2023-03-15",
            "return_date": "2023-03-20",
            "adults": 2,
            "ratings": 4,
            "amenities": [
                "AIR_CONDITIONING"
            ],
            "max": 5
        }
    }
}
Example 2:
{
    "input": "Book a flight from LAX to JFK on June 1st with non-stop preference and a maximum price of $2000, along with hotel reservations at a rating of 4 out of 5 and amenities like swimming pool and wifi.",
    "output": {
       